# Personal Chef

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage

In [34]:
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web for information"""
    
    return tavily_client.search(
        query,
        search_depth="basic",
        max_results=3,
        include_answer=True,
    )

In [35]:
system_prompt = """
You are a personal Ecuadorian chef. The user will give you a list of ingredients thay have left over in their house.
Using the web search tool, search the web for recipes that can be made with the ingredients they have.
Return recipe suggestions and eventually the recipe instructions to the user, if requested.
"""

In [36]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [37]:
config = {
    "configurable": {"thread_id": "1"},
    "recursion_limit": 5,
}

response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover chicken and rice. What can I make?")]},
    config
)

print(response['messages'][-1].content)

Nice! Leftover chicken and rice open up some easy, comforting options. Here are a few ideas you can make with what you have (from simple to a bit more loaded):

- Chicken and Rice Casserole (one-dish bake)
  - What it is: A creamy, cheesy bake using cooked chicken and rice with veggies like peas.
  - Why you’ll like it: Uses leftovers, cooks in one pan, and you can swap in whatever veggies you have (peas, broccoli, corn).
  - Quick note: Often includes a creamy sauce and cheese on top.

- Leftover Chicken Fried Rice
  - What it is: A quick stir-fry using day-old rice, shredded chicken, eggs, and whatever veggies you’ve got.
  - Why you’ll like it: Fast, flexible, and great for repurposing leftovers; you’ll get nice, slightly crispy rice if you push the rice a bit in the pan.
  - Quick note: Season with soy or tamari, garlic, and a splash of sesame oil if you have it.

- Chicken Rice Pilaf
  - What it is: Sautéed onions/garlic, add the rice and chicken, and cook with broth or water unti

In [38]:
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='7d7998b1-2913-40aa-b728-154198fa6245'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 604, 'prompt_tokens': 201, 'total_tokens': 805, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DCzPi96X4QuVarVbUE88yguQwDEYw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c92bc-f12f-7651-8101-2bed2641b5ac-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'recipes using leftover chicken and rice'}, 'id': 'call_470X0XqMVNp2iSEbIVe3nCRy', 'type': 'tool_call'}], i

## Image Input

In [39]:
from ipywidgets import FileUpload
from IPython.display import display

import base64

In [40]:
uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [41]:
print(uploader.value)

({'name': 'refrigerator.png', 'type': 'image/png', 'size': 73816, 'content': <memory at 0x11594db40>, 'last_modified': datetime.datetime(2026, 2, 25, 2, 30, 41, 436000, tzinfo=datetime.timezone.utc)},)


In [42]:
uploaded_file = uploader.value[0]
content_mv = uploaded_file["content"]
img_bytes = bytes(content_mv)

In [43]:
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [44]:
multimodal_question = HumanMessage(
    content=[
        {"type": "text", "text": "I am trying to lose weight. What can I make?"},
        {"type": "image", "base64": img_b64, "mime_type": "image/png"}
    ]
)

In [45]:
for event in agent.stream(
    {"messages": [multimodal_question]},
    config=config,
    stream_mode="updates"
):
    print(event)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 846, 'prompt_tokens': 2296, 'total_tokens': 3142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DCzQR063oYztqBiqWwVtXv1Zgps8r', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c92bd-a042-78d1-bafc-2a1db50d6a3e-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'healthy ways to use leftover chicken and rice weight loss'}, 'id': 'call_ruCeslQu0RcwbUMjDtMc6en9', 'type': 'tool_call'}, {'name': 'web_search', 'args': {'parameters': {'query': 'low calorie chicken and vegetable stir fry weight loss'}, 'ignore_cache': False}, 'id': 'call_VC3

In [46]:
response = agent.invoke(
    {"messages": [multimodal_question]},
    config=config
)

print(response["messages"][-1].content)

Great idea—weight-loss friendly meals that use leftover chicken and rice. Here are 3 easy options with a light, flavorful twist you can adapt to what you have. I’ve included simple, Ecuadorian-inspired touches (cilantro, lime, ají) you can add if you like.

1) Cilantro-Lime Chicken & Rice Salad (2 servings)
- Why it’s good: Plenty of veggies and protein, light dressing, keeps you full without heavy sauces.
- Ingredients:
  - 2 cups shredded leftover chicken
  - 1 cup cooked rice (preferably brown rice; otherwise use whatever you have)
  - 4 cups mixed greens (lettuce, spinach, arugula)
  - 1 cup cherry tomatoes, halved
  - 1/4 cup red onion, thinly sliced
  - 1/2 avocado (optional)
  - 1/4 cup fresh cilantro, chopped
  - Juice of 1-2 limes
  - 2 tablespoons plain Greek yogurt or light yogurt (optional, for dressing)
  - Salt and pepper to taste
  - Optional Ecuadorian touch: a pinch of ají lime or a small squeeze of hot pepper juice
- Method:
  1) Toss greens, tomatoes, onion, cilantro

In [47]:
pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='7d7998b1-2913-40aa-b728-154198fa6245'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 604, 'prompt_tokens': 201, 'total_tokens': 805, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DCzPi96X4QuVarVbUE88yguQwDEYw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c92bc-f12f-7651-8101-2bed2641b5ac-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'recipes using leftover chicken and rice'}, 'id': 'call_470X0XqMVNp2iSEbIVe3nCRy', 'type': 'tool_call'}], i